In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import warnings

warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_columns', 500)
%matplotlib inline

# 1. Data Load

In [2]:
target_original = pd.read_csv('data/target_v1.csv')
weather_original = pd.read_csv('data/weather_v1.csv')
hourly_smp_original = pd.read_csv('data/hourly_smp_v1.csv')

target = target_original.copy()
weather = weather_original.copy()
hourly_smp = hourly_smp_original.copy()

In [3]:
oil_price_cl_original = pd.read_csv('result/oil_price_cl.csv')
oil_price_du_original = pd.read_csv('result/oil_price_du.csv')
oil_price_brt_original = pd.read_csv('result/oil_price_brt.csv')

oil_price_cl = oil_price_cl_original.copy()
oil_price_du = oil_price_du_original.copy()
oil_price_brt = oil_price_brt_original.copy()

In [4]:
area_list = weather['area'].unique()
for ls in area_list :
    null_temp  = weather[weather['area'] == ls]['temp'].isnull().sum()
    print('{} total shape : {}'.format(ls, len(weather[weather['area'] == ls]['temp'])))
    print('{} null shape : {}'.format(ls, null_temp))

184 total shape : 17518
184 null shape : 2
185 total shape : 17512
185 null shape : 20
188 total shape : 17518
188 null shape : 7
189 total shape : 17516
189 null shape : 11
328 total shape : 17488
328 null shape : 72
329 total shape : 17514
329 null shape : 5
330 total shape : 17432
330 null shape : 4
724 total shape : 17423
724 null shape : 7
725 total shape : 17469
725 null shape : 11
726 total shape : 17392
726 null shape : 108
727 total shape : 17517
727 null shape : 4
751 total shape : 17438
751 null shape : 12
752 total shape : 17518
752 null shape : 21
753 total shape : 17287
753 null shape : 15
779 total shape : 17518
779 null shape : 3
780 total shape : 17244
780 null shape : 53
781 total shape : 17399
781 null shape : 22
782 total shape : 17515
782 null shape : 3
792 total shape : 17514
792 null shape : 9
793 total shape : 17467
793 null shape : 132
855 total shape : 17448
855 null shape : 503
861 total shape : 17510
861 null shape : 3
862 total shape : 17005
862 null shape 

In [5]:
hourly_smp.shape

(17520, 2)

In [6]:
weather[weather['area'] == 884].head()

,area,datetime,temp,temp_QCFlag,prec,prec_QCFlag,ws,ws_QCFlag,wd,wd_QCFlag,humid,humid_QCFlag,vaporP,dew,landP,landP_QCFlag,seaP,seaP_QCFlag,suntime,suntime_QCFlag,rad,snow,3hr_snow,C,lowmiddleC,typeC,ceiling,vis,sfctype,weathernumber,sfctemp,sfctemp_QCFlag,landtemp_5cm,landtemp_10cm,landtemp_20cm,landtemp_30cm,station
574167,884,2018-02-01 01:00,3.3,NaN,0.0,NaN,1.3,NaN,194.3,NaN,79.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
574168,884,2018-02-01 02:00,2.2,NaN,0.0,NaN,0.9,NaN,229.9,NaN,86.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
574169,884,2018-02-01 03:00,3.8,NaN,0.0,NaN,1.9,NaN,200.2,NaN,79.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
574170,884,2018-02-01 04:00,0.7,NaN,0.0,NaN,1.0,NaN,236.1,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
574171,884,2018-02-01 05:00,2.4,NaN,0.0,NaN,0.9,NaN,189.4,NaN,80.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS


In [7]:
weather[weather['area'] == 884].tail()

,area,datetime,temp,temp_QCFlag,prec,prec_QCFlag,ws,ws_QCFlag,wd,wd_QCFlag,humid,humid_QCFlag,vaporP,dew,landP,landP_QCFlag,seaP,seaP_QCFlag,suntime,suntime_QCFlag,rad,snow,3hr_snow,C,lowmiddleC,typeC,ceiling,vis,sfctype,weathernumber,sfctemp,sfctemp_QCFlag,landtemp_5cm,landtemp_10cm,landtemp_20cm,landtemp_30cm,station
591680,884,2020-01-31 19:00,5.7,NaN,0.0,NaN,1.3,NaN,340.1,NaN,74.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
591681,884,2020-01-31 20:00,6.2,NaN,0.0,NaN,1.2,NaN,305.8,NaN,75.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
591682,884,2020-01-31 21:00,4.9,NaN,0.0,NaN,0.9,NaN,328.0,NaN,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
591683,884,2020-01-31 22:00,3.6,NaN,0.0,NaN,0.9,NaN,306.5,NaN,89.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS
591684,884,2020-01-31 23:00,4.6,NaN,0.0,NaN,1.2,NaN,314.3,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWS


In [8]:
hourly_smp.head()

,datetime,smp
0,2018-02-01 01:00,116.84
1,2018-02-01 02:00,116.84
2,2018-02-01 03:00,116.84
3,2018-02-01 04:00,116.84
4,2018-02-01 05:00,116.84


In [9]:
hourly_smp.tail()

,datetime,smp
17515,2020-01-31 20:00,151.66
17516,2020-01-31 21:00,123.85
17517,2020-01-31 22:00,205.89
17518,2020-01-31 23:00,198.24
17519,2020-01-31 24:00,123.03


**daily smp data 수집 기간**  
**2018년 2월 1일 ~ 2020년 1월 31일**

**데이터 분석 결과 온도를 제외하고는 날씨 데이터에서는 유의미한 변수가 없는것으로 판단**  
**weather data 에서는 temp column만을 추출**

**oil data 수집기간**  
**2006년 4월 18일 ~ 2020년 5월 11일(주말 제외)**

# 2. EDA

### weather dataset을 비롯한 모든 데이터의 필요없는 column을 drop하고 변수명을 변환

In [10]:
weather = weather[weather['area'] == 884]
weather = weather.iloc[:, 1:3]
weather.columns = ['date', 'temp']
hourly_smp.columns = ['date', 'smp']

oil_price_cl = oil_price_cl.iloc[:, 1:3]
oil_price_du = oil_price_du.iloc[:, 1:3]
oil_price_brt = oil_price_brt.iloc[:, 1:3]
oil_price_cl.columns = ['date', 'cl_price']
oil_price_du.columns = ['date', 'du_price']
oil_price_brt.columns = ['date', 'brt_price']

In [11]:
print('weather shape : {}'.format(weather.shape))
print('hourly_smp shape : {}'.format(hourly_smp.shape))
print('oil_price_du shape : {}'.format(oil_price_du.shape))

weather shape : (17518, 2)
hourly_smp shape : (17520, 2)
oil_price_du shape : (3485, 2)


### 각 table의 날짜 데이터를 date format으로 변환

In [12]:
target['date'] = pd.to_datetime(target['date'])

# oil price dataframe에서 날짜를 date column으로 변환
for table in ([oil_price_du, oil_price_cl, oil_price_brt]) : 
    table['date'] = pd.to_datetime(table['date'])
    table.sort_values(by = ['date'], inplace = True)

In [21]:
# weather과 hourly_smp 데이터의 date, time 분해 진행

def date_time_split(data) : 
    time = data['date']
    
    date_list = []
    year = []
    month = []
    day = []
    hour_list = []

    for time_data in time :
        date = time_data.split(' ')[0]
        hour = time_data.split(' ')[1].split(':')[0]

        d = datetime.strptime(date, '%Y-%m-%d')
        date_list.append(pd.to_datetime(date))
        year.append(int(d.year))
        month.append(int(d.month))
        day.append(int(d.day))

        if hour != '24' :
            hour_list.append(int(hour))
        else :
            hour_list.append(int('00'))

    time_dataframe = pd.DataFrame({'date' : date_list, 'year' : year, 'month' : month, 'day' : day, 'hour' : hour_list})
    time_dataframe = time_dataframe.set_index('date') 
    data = data.set_index('date')
    dataframe= pd.merge(data, time_dataframe, on = ['date'])
    
    return dataframe

In [22]:
weather = date_time_split(weather)
hourly_smp = date_time_split(hourly_smp)
weather = weather.iloc[:, 1:]
hourly_smp = hourly_smp.iloc[:, 1:] 

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

**2018년 2월 1일 이전의 데이터는 모두 drop**

In [ ]:
# target data의 날짜에 맞춰 2018년 2월 1일 이후의 데이터만을 sampling
oil_price_du = oil_price_du[oil_price_du['date'] >= '2018-02-01']
oil_price_cl = oil_price_cl[oil_price_cl['date'] >= '2018-02-01']
oil_price_brt = oil_price_brt[oil_price_brt['date'] >= '2018-02-01']

In [ ]:
fig, ax = plt.subplots(figsize = (25,10))

sns.lineplot(x = oil_price_du['du_price'], y = target['smp_mean'], ax = ax, label = 'du_price')
sns.lineplot(x = oil_price_cl['cl_price'], y = target['smp_mean'], ax = ax, label = 'cl_price')
sns.lineplot(x = oil_price_brt['brt_price'], y = target['smp_mean'], ax = ax, label = 'brt_price')
plt.xlim(20, 90)
ax.legend()
plt.show()

In [ ]:
print('weather shape : {}'.format(weather.shape))
print('hourly_smp shape : {}'.format(hourly_smp.shape))
print('oil_price_du shape : {}'.format(oil_price_du.shape))

In [ ]:
weather.head()

### smp, supply, oil_pirce, weather를 합친 train 제작

In [ ]:
train = hourly_smp.copy()

for table in ([oil_price_du, oil_price_cl, oil_price_brt, weather]) :
    train = pd.merge(train, table, on = ['date'])

train.head(5)

In [ ]:
valuable_column = ['smp', 'date', 'du_price', 'cl_price', 'brt_price', 'temp', 'year_y', 'month_y', 'day_y', 'hour_y']
train = train[valuable_column]

## train data의 feature간 상관관계 분석

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))

sns.heatmap(train.corr(), annot = True, ax = ax, fmt = '.2f')

# 3. Machine learning modelling 

#### LSTM on keras

**temp column에 null값이 존재**  
**월별 평균 온도로 null값을 대체**  

In [ ]:
train.columns = ['smp', 'date', 'du_price', 'cl_price', 'brt_price', 'temp', 'year', 'month', 'day', 'hour']
train.head()

In [ ]:
column = train.columns
for col in column :
    null = train[col].isnull().sum()
    print('{} null value : {}'.format(col, null))

In [ ]:
null_month = train[train['temp'].isnull()]['month']
null_month.value_counts()

In [ ]:
null_temp = [0]*12
for month in range(1, 13) :
    null_temp[month-1] = round(train[train['month'] == month]['temp'].mean(), 1)
    
null_temp

In [ ]:
null_temp_dict = {'1': null_temp[0], '2' : null_temp[1], '3' : null_temp[2], '4' : null_temp[3], '5' : null_temp[4], '6' : null_temp[5],
                 '7' : null_temp[6], '8' : null_temp[7], '9' : null_temp[8], '10' : null_temp[9],
                 '11' : null_temp[10], '12' : null_temp[11]}

In [ ]:
for month in (train[train['temp'].isnull() == True]) :
    if train['month'] == 

In [ ]:
null_month = train[train['temp'].isnull()]['month']
null_month_list = []

for month in null_month :
    null_month_list.append(round(train[train['month'] == month]['temp'].mean(), 1))

train[train['temp'].isnull()]['temp'] = null_month_list

In [ ]:
train.info()

In [ ]:
print(len(train[train['temp'].isnull()]['temp']))
print(len(null_month_list))

### 두가지 모델로 나눔
#### 1) 이전의 smp 데이터를 바탕으로 다음 smp 예측
#### 2) 기상 데이터 및 원유가격 데이터를 바탕으로 smp 예측